# Study morphology


### Setup

activate spikeinterf...

In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import neurom as nm
from neurom.view import plotly_impl

import spikeinterface as si

# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023/"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params

data_conf, param_conf = get_config("silico_neuropixels", "concatenated").values()
SORTING_TRUE_PATH = data_conf["sorting"]["simulation"]["ground_truth"]["output"]


FEATURES = [
    "total_length",
    "total_height",
    "total_area",
    "number_of_sections",
    "section_lengths",
]
NEURITES = {
    "apical": nm.core.types.NeuriteType.apical_dendrite,
    "basal": nm.core.types.NeuriteType.basal_dendrite,
    "axon": nm.core.types.NeuriteType.axon,
}

2024-09-25 15:45:22,395 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 15:45:22,445 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [2]:
# load parameters from the first simulation "BlueConfig" file
simulation_params = load_campaign_params(data_conf)

In [24]:
# get circuit
circuit = simulation_params["circuit"]

# get target units
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)


def get_exc_morph_paths(circuit):
    df = circuit.cells.get(SortingTrue.unit_ids, properties=["mtype", "morphology"])
    df.drop_duplicates(subset="morphology", ignore_index=True, inplace=True)

    df["morph_path"] = (
        df["morphology"]
        .apply(lambda row: os.path.join(circuit.config["morphologies"], "%s.asc" % row))
        .astype(str)
    )
    df.drop("morphology", axis=1, inplace=True)
    morph_paths = df.groupby("mtype")["morph_path"].apply(list)

    # get rid of INTs (because of categorical objects...)
    morph_paths = morph_paths[morph_paths.apply(len) > 0]
    return morph_paths


def get_features(morph_paths):
    """Get morphometric features for all morphologies passed"""
    features = {"mtype": [], "feature_name": [], "feature_val": [], "loc": []}
    for mtype, paths in tqdm(morph_paths.items(), desc="mtypes"):
        for path in tqdm(
            paths, desc="morphologies", miniters=len(paths) / 100, leave=False
        ):
            morph = nm.load_morphology(path)
            for neurite_name, neurite_id in NEURITES.items():
                for feature in FEATURES:
                    features["mtype"].append(mtype)
                    feature_name = (
                        "avg_section_length"
                        if feature == "section_lengths"
                        else feature
                    )
                    features["feature_name"].append(feature_name)
                    feature_val = nm.get(feature, morph, neurite_type=neurite_id)
                    feature_val = (
                        np.mean(feature_val)
                        if feature == "section_lengths"
                        else feature_val
                    )
                    features["feature_val"].append(feature_val)
                    features["loc"].append(neurite_name)
    df = pd.DataFrame.from_dict(features)
    df.drop(
        df.loc[(df["mtype"] == "L4_SSC") & (df["loc"] == "apical")].index, inplace=True
    )  # clear L4_SSC apicals...
    return df

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/envs/spikinterf0_100_5/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


In [27]:
# 0.5s
morph_paths = get_exc_morph_paths(circuit)
morph_paths

mtype
L1_NGC-DA    [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L1_NGC-SA    [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L1_SAC       [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_BP       [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_BTC      [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_CHC      [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_LBC      [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_MC       [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_NBC      [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L23_SBC      [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L2_IPC       [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L2_TPC:A     [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L2_TPC:B     [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L3_TPC:A     [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L3_TPC:C     [/gpfs/bbp.cscs.ch/project/proj83/morphologies...
L4_BP        [/gpfs/bbp.cscs.ch/project/proj83/mo

### Get features

In [26]:
df = get_features(morph_paths)
df

,mtype,feature_name,feature_val,loc
0,L1_NGC-DA,total_length,0.000000,apical
1,L1_NGC-DA,total_height,0.000000,apical
2,L1_NGC-DA,total_area,0.000000,apical
3,L1_NGC-DA,number_of_sections,0.000000,apical
4,L1_NGC-DA,avg_section_length,NaN,apical
...,...,...,...,...
20410,L6_UPC,total_length,14528.666153,axon
20411,L6_UPC,total_height,1960.615967,axon
20412,L6_UPC,total_area,7126.851915,axon
20413,L6_UPC,number_of_sections,59.000000,axon


### Plot

In [30]:
astrocyte = nm.load_morphology(morph_paths[0][0])
fig, ax = plotly_impl.plot_morph3d(astrocyte, inline=True)

### References

https://github.com/BlueBrain/NeuroM/blob/master/tutorial/astrocyte_analysis.ipynb